# Demonstrate ``TuneAxis``

In this example, we demonstrate the `apstools.plans.TuneAxis()` plan.  The `TuneAxis()` support may be used to align (a.k.a. *tune*) a signal against an axis.

We'll use a software-only (not connected to hardware) motor as a positioner.  Here, we prepare a signal that is a computation based on the value of our positioner.  The computed signal is a model of a realistic diffraction peak ([pseudo-Voigt](https://en.wikipedia.org/wiki/Voigt_profile), a mixture of a Gaussian and a Lorentzian) one might encounter in a powder diffraction scan.  The model peak is a pseudo-voigt function to which some noise has been added.  Random numbers are used to modify the ideal pseudo-voigt function so as to simulate a realistic signal.

For this demo, we do not need the databroker since we do not plan to review any of this data after collection.  We'll display the data during the scan using the *LiveTable()* code.

In [1]:
from apstools.devices import SynPseudoVoigt
from apstools.plans import TuneAxis
from bluesky import RunEngine
from bluesky import plans as bp
from bluesky.callbacks import LiveTable
import numpy as np
from ophyd import EpicsMotor

RE = RunEngine({})


Figure out which workstation we are running.  The *mint-vm* host has a different IOC prefix.

In [2]:
IOC = "gp:"

Connect to our motor *before* we create the simulated detector signal.

In [3]:
m1 = EpicsMotor(f"{IOC}m1", name="m1")
m1.wait_for_connection()

Define a starting position, we'll use this later in the demo.

In [4]:
m1.move(-1.5)
starting_position = m1.position

## Setup the simulated detector signal.  

Make a simple scan with a simulated motor and the `SynPseudoVoigt()` signal.

In [5]:
from ophyd.sim import motor

det = SynPseudoVoigt('det', motor, 'motor',
    center=0, eta=0.5, scale=1, sigma=1, bkg=0)

live_table = LiveTable(["motor", "det"])

RE(bp.scan([det], motor, -3, 3, 19), live_table)



+-----------+------------+------------+------------+
|   seq_num |       time |      motor |        det |
+-----------+------------+------------+------------+
|         1 | 15:32:07.1 |     -3.000 |      0.056 |
|         2 | 15:32:07.1 |     -2.667 |      0.076 |
|         3 | 15:32:07.1 |     -2.333 |      0.110 |
|         4 | 15:32:07.1 |     -2.000 |      0.168 |
|         5 | 15:32:07.1 |     -1.667 |      0.257 |
|         6 | 15:32:07.1 |     -1.333 |      0.386 |
|         7 | 15:32:07.1 |     -1.000 |      0.553 |
|         8 | 15:32:07.1 |     -0.667 |      0.747 |
|         9 | 15:32:07.1 |     -0.333 |      0.923 |
|        10 | 15:32:07.1 |      0.000 |      1.000 |
|        11 | 15:32:07.1 |      0.333 |      0.923 |
|        12 | 15:32:07.1 |      0.667 |      0.747 |
|        13 | 15:32:07.1 |      1.000 |      0.553 |
|        14 | 15:32:07.1 |      1.333 |      0.386 |
|        15 | 15:32:07.1 |      1.667 |      0.257 |
|        16 | 15:32:07.1 |      2.000 |     

('2d2bf557-4929-4ed3-8112-02de3e9b79ba',)

Make a new signal with randomized values so that we have something interesting to find with `TuneAxis()`.

In [6]:
spvoigt = SynPseudoVoigt(
    'spvoigt', m1, 'm1', 
    center=-1.5 + 0.4*np.random.uniform(), 
    eta=0.2 + 0.5*np.random.uniform(), 
    sigma=0.001 + 0.05*np.random.uniform(), 
    scale=1e5,
    bkg=0.01*np.random.uniform())

Reveal the actual values.  These are the answers we expect to discover.

In [7]:
print(f"{spvoigt.scale = }")
print(f"{spvoigt.center = }")
print(f"{spvoigt.sigma = }")
print(f"{spvoigt.eta = }")
print(f"{spvoigt.bkg = }")

spvoigt.scale = 100000.0
spvoigt.center = -1.3925351184258767
spvoigt.sigma = 0.023117195475099467
spvoigt.eta = 0.5002409281255817
spvoigt.bkg = 0.004540805634184992


We will add the actual values as metadata to these scans.

In [8]:
md = dict(
    activity = "TuneAxis development and testing",
    peak_model = "pseudo Voigt",
    peak_scale = spvoigt.scale,
    peak_center = spvoigt.center,
    peak_sigma = spvoigt.sigma,
    peak_eta = spvoigt.eta,
    peak_bkg = spvoigt.bkg
    )

## Set up the tuner

Create a *TuneAxis()* object.  The *tuner* needs to know the positioner, what range to scan to find the peak, *and* it needs the name of the signal to be scanned (since the signal list may have more than one signal).

In [9]:
tuner = TuneAxis([spvoigt], m1, signal_name=spvoigt.name)
tuner.width = 2.5
tuner.step_factor = tuner.num/2.5

Reconfigure the *LiveTable* to also show the simulated detector signal.

In [10]:
live_table = LiveTable(["m1", "spvoigt"])

# Multi-pass tune

Execute multiple passes to refine the centroid determination.
Each subsequent pass will reduce the width of the next scan by ``step_factor``.

In [11]:
RE(tuner.multi_pass_tune(), live_table, md=md)



+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |    spvoigt |
+-----------+------------+------------+------------+
|         1 | 15:32:09.0 |   -2.75000 |    468.584 |
|         2 | 15:32:09.4 |   -2.47200 |    477.012 |
|         3 | 15:32:09.9 |   -2.19400 |    495.664 |
|         4 | 15:32:10.4 |   -1.91700 |    551.081 |
|         5 | 15:32:10.9 |   -1.63900 |    890.331 |
|         6 | 15:32:11.4 |   -1.36100 |  37649.129 |
|         7 | 15:32:11.9 |   -1.08300 |    731.549 |
|         8 | 15:32:12.4 |   -0.80600 |    531.667 |
|         9 | 15:32:12.9 |   -0.52800 |    489.822 |
|        10 | 15:32:13.4 |   -0.25000 |    474.551 |
+-----------+------------+------------+------------+
generator TuneAxis.multi_pass_tune ['a4114e5d'] (scan num: 2)


PeakStats
================ =================================
key              result                           
================ =================================
x                m1          

('a4114e5d-57db-43f6-88e2-bafd54bb946b',
 '3d6676bd-6314-4075-ae1f-5695fa4fb910',
 'c1906b84-52b0-443b-b184-f27a7aefdfb5',
 '3d930020-ea23-437e-aef3-5d1c8f614777')

Show the results from the multi-pass tuning.

In [12]:
print("final: ", tuner.center)
print("max", tuner.peaks.max)
print("min", tuner.peaks.min)
for stat in tuner.stats:
    print("--", stat.cen, stat.fwhm)
print(f"{m1.position=}  det={spvoigt.get()}")

final:  -1.3925561295137476
max (-1.387, 96329.46348824805)
min (-1.445, 12391.645821650982)
-- Signal(name='PeakStats_cen', parent='PeakStats', value=-1.3613023467014518, timestamp=1638221533.4918845) Signal(name='PeakStats_fwhm', parent='PeakStats', value=0.2805848988995139, timestamp=1638221533.4918973)
-- Signal(name='PeakStats_cen', parent='PeakStats', value=-1.3922730855240273, timestamp=1638221538.3189716) Signal(name='PeakStats_fwhm', parent='PeakStats', value=0.06724971768409405, timestamp=1638221538.3190074)
-- Signal(name='PeakStats_cen', parent='PeakStats', value=-1.3925866293523317, timestamp=1638221541.0214105) Signal(name='PeakStats_fwhm', parent='PeakStats', value=0.051786583357105176, timestamp=1638221541.0214403)
-- Signal(name='PeakStats_cen', parent='PeakStats', value=-1.3925561295137476, timestamp=1638221543.5190797) Signal(name='PeakStats_fwhm', parent='PeakStats', value=0.04809881301054353, timestamp=1638221543.5190878)
m1.position=-1.393  det=12391.645821650982


Compare the final position (just printed) with the expected value shown a couple steps back.

## Single-pass tune

Repeat but with only one pass.  Reset the motor to the starting position and increase the number of steps by a factor of three.

In [13]:
m1.move(starting_position)
tuner.num *= 3
RE(tuner.tune(), live_table, md=md)



+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |    spvoigt |
+-----------+------------+------------+------------+
|         1 | 15:32:26.0 |   -2.75000 |    468.584 |
|         2 | 15:32:26.3 |   -2.66400 |    470.611 |
|         3 | 15:32:26.6 |   -2.57800 |    473.096 |
|         4 | 15:32:26.9 |   -2.49100 |    476.226 |
|         5 | 15:32:27.2 |   -2.40500 |    480.146 |
|         6 | 15:32:27.5 |   -2.31900 |    485.206 |
|         7 | 15:32:27.8 |   -2.23300 |    491.897 |
|         8 | 15:32:28.1 |   -2.14700 |    501.001 |
|         9 | 15:32:28.4 |   -2.06000 |    514.014 |
|        10 | 15:32:28.7 |   -1.97400 |    533.024 |
|        11 | 15:32:29.0 |   -1.88800 |    562.743 |
|        12 | 15:32:29.3 |   -1.80200 |    613.021 |
|        13 | 15:32:29.6 |   -1.71600 |    708.285 |
|        14 | 15:32:29.9 |   -1.62900 |    927.652 |
|        15 | 15:32:30.2 |   -1.54300 |   1607.658 |
|        16 | 15:32:30.5 |   -1.45700 |   71

('b3666b8f-df97-4f76-b353-c0f7f7716f99',)

Compare the single-pass scan with the previous multi-pass scan.  Each used the same number of points overall.  

The results are comparable but we already knew the position of the peak approximately.

In [14]:
print("final: ", tuner.center)
print("max", tuner.peaks.max)
print("min", tuner.peaks.min)
print("centroid", tuner.peaks.cen)
print("FWHM", tuner.peaks.fwhm)
print(f"{m1.position=}  det={spvoigt.get()}")

final:  -1.372677305136548
max (-1.371, 59619.401745892595)
min (-2.75, 468.58384819985065)
centroid -1.372677305136548
FWHM 0.09364757525546175
m1.position=-1.373  det=474.5512608494155
